In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
DRIVE_PATH = '/content/gdrive/MyDrive/question_answering_data/'

In [3]:
import os
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
import regex as re
import json
from glob import glob
import re
from nltk import word_tokenize as lib_tokenizer
import string
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
with open(DRIVE_PATH + 'zac2022_train_merged_final.json') as f:
  data = json.load(f)['data']
  data = [x for x in data if x["category"] == "FULL_ANNOTATION" and "wiki/" in x["answer"]]
  data = pd.DataFrame(data)

data

,id,question,title,text,short_candidate_start,short_candidate,answer,category,is_long_answer
0,c926e7b0717202618a10dd907d4b4c39,Đất nước nào không có quân đội,,"có 23 quốc gia không có lực lượng quân đội, ba...",53,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",wiki/Danh_sách_quốc_gia_không_có_lực_lượng_vũ_...,FULL_ANNOTATION,True
1,82396a18fa9812bfec4d3ecb7ae60905,Núi nào cao nhất châu âu,Shkhara,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,73,núi Elbrus,wiki/Elbrus,FULL_ANNOTATION,True
2,ee8a272bcefac7b96037a1571984b1f8,Thành phố nào là thủ phủ của Ai Cập trong đế q...,Alexandria,"Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , v...",48,Alexandria,wiki/Alexandria,FULL_ANNOTATION,True
3,c51604b25a641b2665c4ce1784b8cc6a,Ai là người đứng đầu trong cuộc chống lại chín...,Cổ Am,Các nghiên cứu lịch sử cho thấy Cổ Am rất có t...,65,Lê Chân,wiki/Lê_Chân,FULL_ANNOTATION,True
4,04bf774a424133d27c34895960937b87,Nền văn hoá Phục Hưng bắt nguồn từ nước nào,Phục Hưng,"Phục Hưng (tiếng Pháp: ""Renaissance"", , , từ ""...",205,Ý,wiki/Ý,FULL_ANNOTATION,True
...,...,...,...,...,...,...,...,...,...
4603,be7f71e3927632f3f3f2afa68ff1b91b,vua nào cho phát hành tiền giấy đầu tiên ở việ...,Tiền tệ Đại Việt thời Trần,Tiền giấy xuất hiện . Năm 1396 thời Trần Thuận...,54,Hồ Quý Ly,wiki/Hồ_Quý_Ly,FULL_ANNOTATION,True
4604,5769cee0487a9674306d271854244b39,Ai là người phát minh ra máy quay phim,,Máy quay phim: Phát minh kỳ diệu của anh em Lu...,37,anh em Lumière Phương Hiền,wiki/Auguste_và_Louis_Lumière,FULL_ANNOTATION,True
4605,e47529f7ade3caaaa60096137798c6ff,tên quận nào ở đà nẵng cũng là tên một danh th...,Ngũ Hành Sơn,Ngũ Hành Sơn hay núi Non Nước là một danh thắn...,285,quận Ngũ Hành Sơn,wiki/Ngũ_Hành_Sơn_(quận),FULL_ANNOTATION,True
4606,935572f75515f8d8ab831648589899c6,trận đánh nào có ý nghĩa quyết định trong khán...,Trận Như Nguyệt,Trận Như Nguyệt là một trận đánh lớn diễn ra ở...,0,Trận Như Nguyệt,wiki/Trận_Như_Nguyệt,FULL_ANNOTATION,True


In [6]:
df_wiki = pd.read_csv(DRIVE_PATH + "processed/wikipedia_20220620_cleaned_v2.csv")
df_wiki

,title,text,bm25_text
0,Trang Chính,"Trang Chính\n\n<templatestyles src=""Wiki2021/s...",trang chính <templatestyles src= wiki2021 styl...
1,Internet Society,Internet Society hay ISOC là một tổ chức quốc...,internet society hay isoc là một tổ chức quốc ...
2,Tiếng Việt,"Tiếng Việt , cũng gọi là tiếng Việt Nam hay Vi...",tiếng việt cũng gọi là tiếng việt nam hay việt...
3,Tiếng Việt,"hệ thống thanh điệu phát triển cao hơn, hệ thố...",hệ thống thanh điệu phát triển cao hơn hệ thốn...
4,Tiếng Việt,tiếp xúc Hán – Việt thành 2 giai đoạn chính: \...,tiếp xúc hán – việt thành 2 giai đoạn chính bu...
...,...,...,...
1944401,VNOI,chuyên viên của các tập đoàn công nghệ lớn như...,chuyên viên của các tập đoàn công nghệ lớn như...
1944402,Vòng hoa,"Vòng hoa (; IPA: ) là một loại hoa, lá, quả, ...",vòng hoa ipa là một loại hoa lá quả cành cây h...
1944403,Vòng hoa,vật trong thần thoại La Mã và Hy Lạp được miêu...,vật trong thần thoại la mã và hy lạp được miêu...
1944404,Vòng hoa,"một vai trò nổi bật, tương tự như ở Anh.\n\n==...",một vai trò nổi bật tương tự như ở anh === biể...


In [7]:
# # Load the BM25 model
# # bm25_model = OkapiBM25Model.load(DRIVE_PATH + 'model/bm25_model.model')

# # Load the BM25 index
# bm25_index = SparseMatrixSimilarity.load(DRIVE_PATH + 'model/bm25_index.index')

# # Load the TF-IDF model
# tfidf_model = TfidfModel.load(DRIVE_PATH + 'model/tfidf_model.model')

# # Load the dictionary
# dictionary = Dictionary.load(DRIVE_PATH + 'model/dictionary.dict')

In [8]:
dict_map = dict({})
def word_tokenize(text):
    global dict_map
    words = text.split()
    words_norm = []
    for w in words:
        if dict_map.get(w, None) is None:
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"')
        words_norm.append(dict_map[w])
    return words_norm

def strip_answer_string(text):
    text = text.strip()
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`':
        if text[0] != '(' and text[-1] == ')' and '(' in text:
            break
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1:
            break
        text = text[:-1].strip()
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`':
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1:
            break
        text = text[1:].strip()
    text = text.strip()
    return text

def strip_context(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    # x = x.replace(",","").replace(".","").replace("?","").replace("!","").replace(")","").replace("(","")
    x = "".join([i for i in x if i not in string.punctuation])
    x = " ".join(x.split()[:128])
    return x

In [9]:
df_wiki['title_lower'] = df_wiki['title'].progress_apply(lambda x: str(x).lower()).progress_apply(post_process)

  0%|          | 0/1944406 [00:00<?, ?it/s]

  0%|          | 0/1944406 [00:00<?, ?it/s]

In [10]:
# corpus = [doc.split() for doc in df_wiki['title_lower']] #simple tokenier


# dictionary = Dictionary(corpus)
# bm25_model = OkapiBM25Model(dictionary=dictionary)
# bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
# bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
# tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')

In [11]:
# dictionary.save("/content/gdrive/MyDrive/question_answering_data/model/bm25_title/dictionary.dict")
# tfidf_model.save("/content/gdrive/MyDrive/question_answering_data/model/bm25_title/tfidf_model.model")
# bm25_index.save("/content/gdrive/MyDrive/question_answering_data/model/bm25_title/bm25_index.index")

In [12]:
# Load the dictionary
dictionary = Dictionary.load("/content/gdrive/MyDrive/question_answering_data/model/bm25_title/dictionary.dict")

# Load the TF-IDF model
tfidf_model = TfidfModel.load("/content/gdrive/MyDrive/question_answering_data/model/bm25_title/tfidf_model.model")

# Load the BM25 index
bm25_index = SparseMatrixSimilarity.load("/content/gdrive/MyDrive/question_answering_data/model/bm25_title/bm25_index.index")

In [13]:
df = pd.DataFrame()
df["answer"] = [post_process(x["short_candidate"]).lower().split() for idx, x in data.iterrows()]

In [14]:
df

,answer
0,"[costa, rica, iceland, panama, micronesia, quầ..."
1,"[núi, elbrus]"
2,[alexandria]
3,"[lê, chân]"
4,[ý]
...,...
4603,"[hồ, quý, ly]"
4604,"[anh, em, lumière, phương, hiền]"
4605,"[quận, ngũ, hành, sơn]"
4606,"[trận, như, nguyệt]"


In [15]:
# del df_wiki

In [16]:
def get_topk(query, topk=4):
    tfidf_query = tfidf_model[dictionary.doc2bow(query)]

    scores = bm25_index[tfidf_query]

    top_titles = np.argsort(scores)[::-1][:topk]

    # top_titles = df_wiki.iloc[top_titles].title.values

    # return top_titles, scores[top_titles]

    return top_titles

all_answer = []
for x in df.answer.values:
  all_answer.append(get_topk(x))

df["top_n"] = all_answer

In [ ]:
df.to_csv(DRIVE_PATH + 'processed/queries_and_topn.csv')

In [17]:
title2idx = dict([(str(x).strip(),y) for x,y in zip(df_wiki.title, df_wiki.index.values)])

In [18]:
topk = 4
total = 0
candidate_ids = []
true_ids = []
for i, row in tqdm(data.iterrows()):
    true_title = row["answer"].replace("_"," ").replace("wiki/","").strip()
    top_n = df.iloc[i].top_n

    try:
      true_idx = title2idx[true_title]
    except:
      print(i)

    true_ids.append(true_idx)
    candidate_ids.append(list(top_n))

0it [00:00, ?it/s]

1173
1341
3194


In [19]:
for x, y in zip(candidate_ids, true_ids):
    if y not in x:
        x[-1] = y
        print(x)

[65002, 65019, 65011, 1527849]
[42785, 42804, 42794, 42854]
[32168, 32169, 32170, 32179]
[11085, 11088, 11086, 11122]
[1908766, 25054, 25048, 25063]
[1557277, 1557276, 1557274, 3660]
[389610, 389609, 389608, 142629]
[78129, 78130, 78131, 78137]
[8030, 8086, 8066, 8127]
[1642441, 1642443, 1642440, 10818]
[2059, 2051, 2053, 2073]
[5349, 5355, 5344, 5435]
[35155, 35190, 35184, 35201]
[48966, 48942, 48960, 48987]
[56694, 56691, 56685, 56695]
[66307, 66317, 66325, 66333]
[179127, 179118, 179128, 179129]
[389230, 389231, 389232, 389235]
[161730, 161729, 161727, 161731]
[55587, 55588, 55596, 55598]
[1555621, 1555620, 1555619, 12515]
[33830, 33843, 33850, 33872]
[594, 624, 613, 638]
[185217, 185214, 185238, 185239]
[1908769, 270698, 270697, 25342]
[488244, 488238, 488240, 488252]
[2576, 2616, 2614, 2643]
[1522914, 1522909, 1522915, 775051]
[1460975, 932673, 1874636, 102032]
[127802, 723388, 723389, 31583]
[26440, 26431, 26439, 26445]
[8030, 8086, 8066, 8127]
[43332, 43348, 43350, 43381]
[99128

In [20]:
data.iterrows()

<generator object DataFrame.iterrows at 0x7f8449dcfdf0>

In [21]:
df = pd.DataFrame()
questions = []
answers = []
titles = []
candidates = []
labels = []
# groups = []
for (idx, sample), true_idx, candidate_idxs in tqdm((zip(data.iterrows(), true_ids, candidate_ids))):
    assert true_idx in candidate_idxs
    question = sample['question']
    answer = sample['short_candidate']
    print(true_idx)
    title = [df_wiki.title.values[i] for i in candidate_idxs]
    candidate = [df_wiki.text.values[i] for i in candidate_idxs]
    label = [1 if x == true_idx else 0 for x in candidate_idxs]

    questions.extend([question,]*len(candidate_idxs))
    answers.extend([answer,]*len(candidate_idxs))
    # groups.extend([idx,]*len(candidate_idxs))
    titles.extend(title)
    candidates.extend(candidate)
    labels.extend(label)

df["question"] = questions
df["answer"] = answers
df["title"] = titles
df["candidate"] = candidates
df["label"] = labels
# df["group"] = groups
df.candidate = df.candidate.progress_apply(lambda x: " ".join(x.strip().split()))

0it [00:00, ?it/s]

1527849
96726
239245
168639
42854
32179
506702
11122
25063
3660
71378
1783577
94491
170173
142629
78137
8127
10818
980
65484
2073
36770
5435
35201
110621
79380
48987
1642462
56695
11051
66333
179129
389235
161731
27886
55598
12515
33872
638
185239
284088
496096
50529
197899
25342
488252
2643
775051
102032
31583
63539
179306
26445
8127
50943
21580
43381
813738
99136
206
16567
2489
186613
16567
18928
14838
1593457
1283
304128
2140
70998
96597
14260
69791
171460
43227
479218
47023
742474
195301
31382
207851
158458
186596
284088
88628
144980
2691
65784
180484
46864
134302
81239
94754
5714
1231606
297975
1530059
110447
5948
137389
26445
432
7102
2643
109337
99485
53558
62903
54539
138266
9405
1597507
1521314
14260
314645
2766
48251
64136
7102
18529
63003
1276468
14838
96892
1276468
31966
86625
69670
116446
306770
92984
166095
95850
1274820
36491
1283
284969
131318
1506079
63031
62859
58206
68682
1588502
4586
80129
11122
1264505
171460
54539
90396
592
54539
69670
157025
503283
4053
33255
468

  0%|          | 0/18432 [00:00<?, ?it/s]

In [22]:
df.to_csv(DRIVE_PATH + 'processed/train_ranking2.csv')

,question,answer,title,candidate,label
0,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Costa Rica,"cai trị của Tây Ban Nha, Costa Rica là một tro...",0
1,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Costa Rica,"công nhân, bằng Abitur của Đức hay High School...",0
2,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Costa Rica,"Rica, có thể chiếm tới 10% trên tổng thu của n...",0
3,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Danh sách quốc gia không có lực lượng vũ trang,động bên ngoài biên giới Nhật Bản bởi Liên Hợp...,1
4,Núi nào cao nhất châu âu,núi Elbrus,Elbrus,sườn dốc của núi nơi kết thúc hệ thống cáp. Tr...,1
...,...,...,...,...,...
18427,trận đánh nào có ý nghĩa quyết định trong khán...,Trận Như Nguyệt,Trận Như Nguyệt,chiến này. BULLET::::- Nhà Lý BULLET::::- Lý T...,1
18428,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,Eros,"ban cho một lời sấm truyền giả: ""Chồng của Psi...",1
18429,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,Eros,"Eros Trong thần thoại Hy Lạp, ""Eros"" là vị thầ...",0
18430,trong thần thoại hy lạp vị thần tình yêu có tê...,Eros,433 Eros,Eros,0


In [24]:
df.to_csv(DRIVE_PATH + 'processed/train_ranking2.csv')

# END